In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **STEP 3: MODEL BUILDING AND SELECTION**

In [ ]:
!pip install colorama

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
plt.style.use('seaborn')
from colorama import Style, Fore, Back
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix



### **PREDICTING SUCCESS OF A TERRORIST ATTACK**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Hamoye/clean_df.csv", index_col=[0])

In [ ]:
# function to display UNIQUE examples of data in each field

def getDesc(data, Column, numExamples=5):
    x = list(set([i for i in data[Column] if i==i]))
    if len(x)<numExamples:
        numExamples = len(x)
    x = list(np.random.choice(x, size=numExamples, replace=False))
    print('Examples in '+Style.BRIGHT+Back.CYAN+Column+Back.RESET+':'+Style.RESET_ALL, x, '\n')
    return None

# ======================================================================================

# function to display the number of missing values in each field

def getNumNan(data, thisFeature):
    ms = data[data[thisFeature] != data[thisFeature]].shape[0]
    print('Missing values in '+Style.BRIGHT+Back.CYAN+thisFeature+Back.RESET+Style.RESET_ALL+': {}'.format(ms))
    return None

In [ ]:
for i in df.columns:
    getNumNan(df, i)
    getDesc(df, i, 7)

Missing values in Year: 0
Examples in Year: [1977, 2004, 1974, 2014, 1999, 2017, 1988] 

Missing values in Month: 0
Examples in Month: [6, 4, 8, 5, 3, 10, 0] 

Missing values in Day: 0
Examples in Day: [15, 11, 8, 18, 9, 27, 21] 

Missing values in Extended: 0
Examples in Extended: [1, 0] 

Missing values in Country: 0
Examples in Country: ['Australia', 'Myanmar', 'Chad', 'Fiji', 'Bulgaria', 'Madagascar', 'Turkmenistan'] 

Missing values in Region: 0
Examples in Region: ['Southeast Asia', 'Sub-Saharan Africa', 'North America', 'Central Asia', 'Middle East & North Africa', 'East Asia', 'Australasia & Oceania'] 

Missing values in City: 0
Examples in City: ['Lachish regional council', 'Ashgabat', 'Athiabari', 'Between Karabudakhkent and Kakashura', 'Pervomayskaya', 'Balghali', 'Quibala'] 

Missing values in Success: 0
Examples in Success: [1, 0] 

Missing values in Suicide: 0
Examples in Suicide: [0, 1] 

Missing values in Attack_Type: 0
Examples in Attack_Type: ['Assassination', 'Facili

In [ ]:
df.describe()

,Year,Month,Day,Extended,Success,Suicide,No_Of_Killed,No_Of_Wounded,Property,country,region,attacktype1,targtype1,natlty1,weaptype1
count,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000,181691.000000
mean,2002.638997,6.467277,15.505644,0.045346,0.889598,0.036507,2.266860,2.883296,-0.544556,131.968501,7.160938,3.247547,8.439719,127.457458,6.447325
std,13.259430,3.388303,8.814045,0.208063,0.313391,0.187549,11.227057,34.309747,3.122889,112.414535,2.933408,1.915772,6.653838,88.949238,2.173435
min,1970.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,4.000000,1.000000,1.000000,1.000000,4.000000,1.000000
25%,1991.000000,4.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,78.000000,5.000000,2.000000,3.000000,83.000000,5.000000
50%,2009.000000,6.000000,15.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,98.000000,6.000000,3.000000,4.000000,101.000000,6.000000
75%,2014.000000,9.000000,23.000000,0.000000,1.000000,0.000000,2.000000,2.000000,1.000000,160.000000,10.000000,3.000000,14.000000,168.000000,6.000000
max,2017.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1570.000000,8191.000000,1.000000,1004.000000,12.000000,9.000000,22.000000,1004.000000,13.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181691 entries, 0 to 181690
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Year                181691 non-null  int64  
 1   Month               181691 non-null  int64  
 2   Day                 181691 non-null  int64  
 3   Extended            181691 non-null  int64  
 4   Country             181691 non-null  object 
 5   Region              181691 non-null  object 
 6   City                181691 non-null  object 
 7   Success             181691 non-null  int64  
 8   Suicide             181691 non-null  int64  
 9   Attack_Type         181691 non-null  object 
 10  Target_Type         181691 non-null  object 
 11  Target_Subtype      181691 non-null  object 
 12  Target              181691 non-null  object 
 13  Target_Nationality  181691 non-null  object 
 14  Attack_Group        181691 non-null  object 
 15  Weapon              181691 non-nul

In [ ]:
df= pd.DataFrame(df)

In [ ]:
cat_col = ['Success','Suicide', 'natlty1', 'weaptype1', 'attacktype1', 'targtype1', 'Year', 'Month', 'Day', 'Extended','Property','country', 'region']
for col in cat_col:
  df[col] = df[col].astype("category")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181691 entries, 0 to 181690
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   Year                181691 non-null  category
 1   Month               181691 non-null  category
 2   Day                 181691 non-null  category
 3   Extended            181691 non-null  category
 4   Country             181691 non-null  object  
 5   Region              181691 non-null  object  
 6   City                181691 non-null  object  
 7   Success             181691 non-null  category
 8   Suicide             181691 non-null  category
 9   Attack_Type         181691 non-null  object  
 10  Target_Type         181691 non-null  object  
 11  Target_Subtype      181691 non-null  object  
 12  Target              181691 non-null  object  
 13  Target_Nationality  181691 non-null  object  
 14  Attack_Group        181691 non-null  object  
 15  Weapon           

In [ ]:
df1 = df.copy()

### **DROPPING UNIMPORTANT FEATURES**

In [ ]:
df.corr()

,No_Of_Killed,No_Of_Wounded
No_Of_Killed,1.000000,0.443326
No_Of_Wounded,0.443326,1.000000


In [ ]:
df1 = df1.drop(columns = ['Month','Day','City','target1', 'gname', 'Country', 'Region','Target', 'Attack_Type','Target_Type', 'Target_Subtype', 'Target','Target_Nationality','Attack_Group', 'Weapon'])


In [ ]:
df1.shape

(181691, 13)

ENCODING CATEGORICAL DATA

In [ ]:
df1

,Year,Extended,Success,Suicide,No_Of_Killed,No_Of_Wounded,Property,country,region,attacktype1,targtype1,natlty1,weaptype1
0,1970,0,1,0,1.0,0.0,0,58,2,1,14,58.0,13
1,1970,0,1,0,0.0,0.0,0,130,1,6,7,21.0,13
2,1970,0,1,0,1.0,0.0,0,160,5,1,10,217.0,13
3,1970,0,1,0,0.0,0.0,1,78,8,3,7,217.0,6
4,1970,0,1,0,0.0,0.0,1,101,4,7,7,217.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,2017,0,1,0,1.0,2.0,-9,182,11,2,4,182.0,5
181687,2017,0,1,0,2.0,7.0,1,200,10,3,4,167.0,6
181688,2017,0,1,0,0.0,0.0,1,160,5,7,14,160.0,8
181689,2017,0,0,0,0.0,0.0,-9,92,6,3,2,92.0,6


In [ ]:
X = df1.loc[:, df1.columns != 'Success']

In [ ]:
y = df["Success"]


In [ ]:
# Split dataset into a training and testing set.
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [ ]:
print ('Number of observations in the target variable before oversampling of the minority class:', np.bincount (y_train) )

ros = RandomOverSampler (random_state=2)
X_train, y_train = ros.fit_resample (X_train, y_train)

print ('\nNumber of observations in the target variable after oversampling of the minority class:', np.bincount (y_train) )

Number of observations in the target variable before oversampling of the minority class: [ 13938 113245]

Number of observations in the target variable after oversampling of the minority class: [113245 113245]


In [ ]:
sc =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
clf = [ LogisticRegression(random_state=2), DecisionTreeClassifier(random_state=2), SVC (random_state=2),
       RandomForestClassifier(random_state=2), GradientBoostingClassifier(random_state=2) ]
models = [ 'Logistic Regression', 'Tree', 'Support vector machine', 'RFC', 'Gradient boost' ]



### **BUILDING A PIPELINE FOR THE MODELLING PROCESS**

In [ ]:
for i in range(0, len(clf)):
        text_clf = Pipeline([('clf', clf[i])])
        print(text_clf)
        text_clf.fit(X_train, y_train)  
        predictions = text_clf.predict(X_test)
        
        print(metrics.confusion_matrix(y_test,predictions))
        print(metrics.classification_report(y_test,predictions))
        print('F1_score: ',round(metrics.f1_score(y_test,predictions, average = 'weighted'),3))
        print('-------------------------------------------------------')

Pipeline(steps=[('clf', LogisticRegression(random_state=2))])
[[ 4515  1606]
 [18692 29695]]
              precision    recall  f1-score   support

           0       0.19      0.74      0.31      6121
           1       0.95      0.61      0.75     48387

    accuracy                           0.63     54508
   macro avg       0.57      0.68      0.53     54508
weighted avg       0.86      0.63      0.70     54508

F1_score:  0.696
-------------------------------------------------------
Pipeline(steps=[('clf', DecisionTreeClassifier(random_state=2))])
[[ 4662  1459]
 [ 2873 45514]]
              precision    recall  f1-score   support

           0       0.62      0.76      0.68      6121
           1       0.97      0.94      0.95     48387

    accuracy                           0.92     54508
   macro avg       0.79      0.85      0.82     54508
weighted avg       0.93      0.92      0.92     54508

F1_score:  0.924
-------------------------------------------------------
Pipeline(s